# Naive Bayes polarity classification
""" AIT726 Final project - Part 1 - Naive Bayes polarity Classification using Naive Bayes Algorithm on SemEval’16 dataset ( 1708 training dataset and 587 testing dataset ) and Foursquare ( 849 testing dataset ) 

Authors: Yasas, Prashanti, Ashwini
 
Command to run the file: run polarity_classification.ipynb

Flow: 
Data loading -  For all the restaurant reviews we parsed the XML files and retrieved the reviews for SemEval’16 and foursquare datasets.

Preprocessing and Feature Extraction - Tokenization, and lemmatization of reviews is performed to get word tokens and their root words. Text vectorization is then performed using term frequency-inverse document frequency (TF-IDF) vectorizer with a ngram range of 1-3. In addition to TF-IDF features, we provided the Aspect Category as a feature when training the polarity classification model. 
 
Baseline Models: Naive Bayes Classifier 

Cross-Validation and Error Analysis: Performed five-fold cross-validation on the training data and performed the error analysis using the predictions obtained in cross-validation. 

Train Models: Trained the models using the optimal hyper-parameters explored in the cross validation process on the whole training dataset. 

Evaluation: Evaluated the trained models on the test data (SemEval’16 and Foursquare datasets)

Note : For more details please check README file
"""

In [2]:
%matplotlib inline

In [3]:
from absa.config import DATA_PATHS
from absa.dataset import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# Pass sentences through spacy nlp pipeline and get the output terms
import spacy
nlp = spacy.load('en')

 """Load and display semeval16 training dataset
   
   Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )
   
   Polarity is also read along with aspect"""
   

In [4]:
train_ds_path = DATA_PATHS['asba.semeval16.raw.train']

df_train = load_dataset(train_ds_path)

df_train = df_train.loc[:, ['id', 'text', 'category', 'polarity']]

df_train = pd.DataFrame({
    'polarity': df_train.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_train.head()

,id,text,category,polarity
0,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,[negative]
1,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,[negative]
2,1004293:2,"They never brought us complimentary noodles, i...",SERVICE#GENERAL,[negative]
3,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,[negative]
4,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#STYLE_OPTIONS,[negative]


# Prepare Labels

 """As we have multiple aspects we have used MultiLabelBinarizer to create y_train"""

In [5]:
mlb = MultiLabelBinarizer()

y_train = mlb.fit_transform(df_train.polarity)

y_train.shape

(2258, 3)

# Extract Features

In [7]:
# Lemmatize input sentence
def lemmatize(x):
    return ' '.join([token.lemma_ for token in nlp(x)])

# Select column from provided dataframe
def select_column(df, column):
    return df.loc[:, column].values

# Reshape array helper function
def reshape_array(array, shape):
    return array.reshape(*shape)

    """ Feature extraction: 
    Select sentences and perform Text Vectorization using TF-IDF (Ngrams considered - Unigram, bigram and trigram)
    Select aspect and perform encoding using OneHotEncoder()
    Later a featureunion of both results are performed """

In [8]:
pipeline = make_union(
    make_pipeline(
        FunctionTransformer(select_column, kw_args={'column': 'text'}),
        TfidfVectorizer(preprocessor=lemmatize, ngram_range=(1, 3)),
    ),
    make_pipeline(
        FunctionTransformer(select_column, kw_args={'column': 'category'}),
        FunctionTransformer(reshape_array, kw_args={'shape': (-1, 1)}),
        OneHotEncoder(),
    ),
)

x_train = pipeline.fit_transform(df_train).toarray()

x_train.shape

(2258, 28603)

# Cross Validatation

""" multiclass/multilabel Gaussian Naive Bayes model is used to train semeval training dataset with 3-fold                     cross validation.
Respective precision_micro, recall_micro and f1_micro scores are presented """

In [9]:
scoring = ['precision_micro', 'recall_micro', 'f1_micro']

clf = OneVsRestClassifier(GaussianNB())

scores = cross_validate(clf, x_train, y_train, scoring=scoring, cv=3)

print('Micro Precision Score\t', np.average(scores['test_precision_micro']))
print('Micro Recall Score\t', np.average(scores['test_recall_micro']))
print('Micro F1 Score\t\t', np.average(scores['test_f1_micro']))

Micro Precision Score	 0.6582456041584567
Micro Recall Score	 0.6447822098633414
Micro F1 Score		 0.6513992369407747


# Error analysis

""" Export results to an excel for performing error Analysis """

In [47]:
from sklearn.model_selection import cross_val_predict

train_pred = cross_val_predict(clf, x_train, y_train, cv=5)

prds = mlb.inverse_transform(train_pred)

df_train['predictions'] = [list(x) for x in prds]

df_train.to_excel('./output/sc_nb.xlsx')

# Train Model

"""Model fit"""

In [10]:
clf.fit(x_train, y_train)

OneVsRestClassifier(estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                    n_jobs=None)

# Evaluate Model

"""Load and display semeval16 testing dataset
   
   Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [11]:
test_ds_path = DATA_PATHS['asba.semeval16.raw.test.gold']

df_test = load_dataset(test_ds_path)

df_test = df_test.loc[:, ['id', 'text', 'category', 'polarity']]

df_test = pd.DataFrame({
    'polarity': df_test.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_test.head()

,id,text,category,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD#QUALITY,[positive]
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD#QUALITY,[positive]
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD#STYLE_OPTIONS,[neutral]
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD#QUALITY,[positive]
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD#QUALITY,[positive]


""" Predict the results on semval2016 testing dataset """

In [12]:
# Testing
x_test = pipeline.transform(df_test).toarray()

y_pred = clf.predict(x_test)

y_pred.shape

(743, 3)

In [13]:
y_true = mlb.transform(df_test.polarity)

y_true.shape

(743, 3)

""" Evaluate the results for semval2016 testing dataset """"

In [14]:
('f1_score', f1_score(y_true, y_pred, average='micro'))

('f1_score', 0.6301188903566711)

# Foursquare dataset

"""Load and display Foursquare testing dataset
   
   Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [ ]:
test_fs_ds_path = DATA_PATHS['asba.foursquare.raw.test.gold']

df_test_fs = load_dataset(test_fs_ds_path)

df_test_fs = df_test_fs.loc[:, ['id', 'text', 'category', 'polarity']]

df_test_fs = pd.DataFrame({
    'polarity': df_test_fs.groupby(['id', 'text', 'category'])['polarity'].apply(list),
}).reset_index()

df_test_fs.head()

""" Predict the results on Foursquare testing dataset """

In [ ]:
# Testing
x_test_fs = pipeline.transform(df_test_fs).toarray()

y_pred_fs = clf.predict(x_test_fs)

y_pred_fs.shape

In [ ]:
y_true_fs = mlb.transform(df_test_fs.polarity)

y_true.shape

""" Evaluate the results for Foursquare testing dataset """"

In [ ]:
('f1_score', f1_score(y_true_fs, y_pred_fs, average='micro'))